In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score

# =====================================================
# 1. Load train / val + Kaggle test.csv (inside analysis/)
# =====================================================

train = pd.read_csv("train_split_merged_expanded_data.csv")
val   = pd.read_csv("val_split_merged_expanded_data.csv")

# Kaggle test template (has id/date/warengruppe; no umsatz)
test  = pd.read_csv("analysis/test_split_merged_data_updated.csv")

# Keep ids for submission
test_ids = test["id"].copy()

# -----------------------------------------------------
# Add exogenous features to Kaggle test.csv
# We'll take them from merged_expanded_data_fullcalendar.csv
# (1 row per date for weather + calendar flags)
# -----------------------------------------------------

features_src = pd.read_csv(
    "merged_expanded_data_fullcalendar.csv",
    parse_dates=["date"]
)

# keep only one row per date (features are date-level)
feat_cols = [
    "date",
    "Temperatur",
    "KielerWoche",
    "school_holiday",
    "public_holiday",
]
feat_cols = [c for c in feat_cols if c in features_src.columns]

features_daily = (
    features_src[feat_cols]
    .drop_duplicates(subset=["date"])
    .sort_values("date")
)

# make sure test has date as datetime
test["date"] = pd.to_datetime(test["date"])

# merge features onto Kaggle test
test = test.merge(features_daily, on="date", how="left")

# Kaggle test has no target -> create it as NaN so concat works
test["umsatz"] = np.nan

# Mark dataset type and concatenate so that lags/rolling
# are computed consistently over time per warengruppe.
train["dataset"] = "train"
val["dataset"]   = "val"
test["dataset"]  = "test"
df_all = pd.concat([train, val, test], ignore_index=True)

# =====================================================
# 2. Basic date & calendar features
# =====================================================

df_all["date"] = pd.to_datetime(df_all["date"])

df_all["Wochentag"] = df_all["date"].dt.day_name()
df_all["Month"]     = df_all["date"].dt.month
df_all["dayofyear"] = df_all["date"].dt.dayofyear  # goes from 1 to 365 then jumps back to 1,
# Additionally a linear model can’t naturally understand that Dec 31 and Jan 1 are “close”. Numerically they look far apart (365 vs 1)
# That's why we use sine and cosine transformations to encode seasonality in a way that reflects the cyclical nature of the data.
# The linear regression can now learn smooth yearly patterns like: high sales in summer, low sales in winter, spikes around certain times of year, etc.

df_all["sin_season"] = np.sin(2 * np.pi * df_all["dayofyear"] / 365)
df_all["cos_season"] = np.cos(2 * np.pi * df_all["dayofyear"] / 365)

df_all["is_weekend"] = df_all["Wochentag"].isin(["Saturday", "Sunday"]).astype(int)

# Integer/calendar flags
for col in ["KielerWoche", "school_holiday", "public_holiday"]:
    if col in df_all.columns:
        df_all[col] = pd.to_numeric(df_all[col], errors="coerce").fillna(0).astype(int)
    else:
        df_all[col] = 0

# =====================================================
# 3. Lag features & rolling statistics per warengruppe
# =====================================================

# Sort so that groupby().shift() and rolling are correct in time
df_all = df_all.sort_values(["warengruppe", "date"])

# Lags (memory of the past) of target (umsatz): daily and weekly patterns
# Lags are used because bakery demand is often autocorrelated; past sales influence future sales.
# e.g. if croissant sales were high yesterday, they are likely to be high today too.
# So with lags , the model also sees "recent demand level"
for lag in [1, 2, 7, 14]:
    df_all[f"lag_{lag}"] = (
        df_all
        .groupby("warengruppe")["umsatz"]
        .shift(lag)
    )

# Rolling mean & std of past sales; smoothes the demand and cature "recent level" and "volatility"
# "shift(1)" to avoid using current day's value in rolling stats, which would be cheating and avoids data leakage.
# Rolling mean says "on average, hoe much has this product group sold recently?": smoothes out day-to-day noise.
# Rolling std says "how much does sales vary recently?" or "How stable is demand recently?": captures volatility/ how noisy the data is.
for window in [7, 14, 30]:
    df_all[f"roll{window}_mean"] = (
        df_all
        .groupby("warengruppe")["umsatz"]
        .shift(1)
        .rolling(window)
        .mean()
    )
    df_all[f"roll{window}_std"] = (
        df_all
        .groupby("warengruppe")["umsatz"]
        .shift(1)
        .rolling(window)
        .std()
    )

# =====================================================
# 4. One-hot encode weekday (on full df), then split back
# =====================================================

df_all = pd.get_dummies(df_all, columns=["Wochentag"], drop_first=True)

train_fe = df_all[df_all["dataset"] == "train"].copy()
val_fe   = df_all[df_all["dataset"] == "val"].copy()
test_fe  = df_all[df_all["dataset"] == "test"].copy()

# =====================================================
# 5. Define feature set
# =====================================================

weekday_cols = [c for c in train_fe.columns if c.startswith("Wochentag_")]

feature_cols = [
    "Temperatur",
    "KielerWoche",
    "school_holiday",
    "public_holiday",
    "Month",
    "sin_season",
    "cos_season",
    "is_weekend",
    "lag_1",
    "lag_2",
    "lag_7",
    "lag_14",
    "roll7_mean",
    "roll7_std",
    "roll14_mean",
    "roll14_std",
    "roll30_mean",
    "roll30_std",
] + weekday_cols

target_col = "umsatz"

# =====================================================
# 5b. NEW: Per-warengruppe feature pruning
# =====================================================

FEATURES_BY_WG = {
    1.0: [c for c in feature_cols if c not in ["Temperatur", "KielerWoche"]],
    2.0: feature_cols,
    3.0: feature_cols,
    4.0: [c for c in feature_cols if c not in ["Temperatur", "KielerWoche"]],
    5.0: feature_cols,  
    6.0: feature_cols, 
}


# =====================================================
# helper: build model pipeline for a given feature list
# =====================================================

def build_model(cols):
    preprocessor = ColumnTransformer(
        transformers=[
            ("num", Pipeline([
                ("imputer", SimpleImputer(strategy="mean"))
            ]), cols),
        ]
    )

    model = Pipeline(steps=[
        ("prep", preprocessor),
        ("reg", LinearRegression())
    ])
    return model

# =====================================================
# 6. Train one LinearRegression model per product group
#    + ANOVA-like variance partitioning
# =====================================================

product_groups = sorted(train_fe["warengruppe"].dropna().unique())
pred_list = []
models_by_wg = {}

# Feature groups for ANOVA-like partitioning (must exist in "features" to be removable)
feature_groups = {
    "Weekday": weekday_cols,
    "School Holiday": ["school_holiday"],
    "Public Holiday": ["public_holiday"],
    "Seasonality": ["sin_season", "cos_season"],
    "Month": ["Month"],
    "KielerWoche": ["KielerWoche"],
    "Temperature": ["Temperatur"],
}

for wg in product_groups:
    print(f"\n==============================")
    print(f" Warengruppe {wg}")
    print("==============================")

    train_wg = train_fe[train_fe["warengruppe"] == wg].copy()
    val_wg   = val_fe[val_fe["warengruppe"] == wg].copy()
    test_wg  = test_fe[test_fe["warengruppe"] == wg].copy()

    features = FEATURES_BY_WG.get(float(wg), feature_cols)

    print(f"Rows: train={len(train_wg)}, val={len(val_wg)}, test={len(test_wg)}")
    print(f"Using {len(features)} features for wg {wg}")

    X_train = train_wg[features]
    y_train = train_wg[target_col]

    X_val = val_wg[features]
    y_val = val_wg[target_col]

    X_test = test_wg[features]

    # --- Full model ---
    full_model = build_model(features)
    full_model.fit(X_train, y_train)
    models_by_wg[wg] = full_model

    # R² on training data
    y_train_pred = full_model.predict(X_train)
    full_r2_train = r2_score(y_train, y_train_pred)
    print(f"Full model R² (train): {full_r2_train:.3f}")

    # R² on validation data
    y_val_pred = full_model.predict(X_val)
    full_r2_val = r2_score(y_val, y_val_pred)
    print(f"Full model R² (val):   {full_r2_val:.3f}")

    # --- ANOVA-like variance partitioning (TRAIN R²) ---
    rows = []
    for group_name, cols_in_group in feature_groups.items():
        cols_in_group = [c for c in cols_in_group if c in features]
        if len(cols_in_group) == 0:
            continue

        reduced_cols = [c for c in features if c not in cols_in_group]
        if len(reduced_cols) == 0:
            continue

        reduced_model = build_model(reduced_cols)
        reduced_model.fit(train_wg[reduced_cols], y_train)

        y_reduced_pred = reduced_model.predict(train_wg[reduced_cols])
        r2_reduced = r2_score(y_train, y_reduced_pred)

        rows.append({
            "Feature Group": group_name,
            "ΔR²": full_r2_train - r2_reduced,
            "Reduced Model R²": r2_reduced
        })

    if len(rows) > 0:
        var_table = (
            pd.DataFrame(rows)
            .sort_values("ΔR²", ascending=False)
            .reset_index(drop=True)
        )
        print("\nANOVA-like variance partitioning:")
        print(var_table.to_string(index=False))

    # --- Predict on TEST ---
    y_test_pred = full_model.predict(X_test)

    pred_list.append(
        pd.DataFrame({
            "id": test_wg["id"].values,
            "umsatz_Prediction": y_test_pred
        })
    )

# =====================================================
# 7. Build submission file
# =====================================================

submission = pd.concat(pred_list, ignore_index=True).sort_values("id")
submission.to_csv("submission_linear_regression_byGulfem_2.csv", index=False)

submission.head()



 Warengruppe 1
Rows: train=1462, val=357, test=355
Using 22 features for wg 1
Full model R² (train): 0.514
Full model R² (val):   0.476

ANOVA-like variance partitioning:
 Feature Group      ΔR²  Reduced Model R²
       Weekday 0.188475          0.325088
Public Holiday 0.034388          0.479174
School Holiday 0.017325          0.496237
   Seasonality 0.012323          0.501239
         Month 0.004358          0.509205

 Warengruppe 2
Rows: train=1462, val=357, test=355
Using 24 features for wg 2
Full model R² (train): 0.854
Full model R² (val):   0.862

ANOVA-like variance partitioning:
 Feature Group      ΔR²  Reduced Model R²
Public Holiday 0.020560          0.833907
       Weekday 0.017006          0.837460
School Holiday 0.004173          0.850294
   KielerWoche 0.003262          0.851204
         Month 0.003241          0.851225
   Seasonality 0.002723          0.851743
   Temperature 0.000166          0.854300

 Warengruppe 3
Rows: train=1462, val=357, test=355
Using 24 feature

,id,umsatz_Prediction
0,1808011,143.314043
355,1808012,584.764245
710,1808013,293.314359
1065,1808014,64.127047
1419,1808015,283.177036
